<a href="https://colab.research.google.com/github/jemmee/tpu/blob/main/tpu_matmul_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 966.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00


In [ ]:
import jax
# This will show 'TpuDevice' if everything is set up correctly
print(jax.devices())

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)]


In [ ]:
import jax
try:
    # This should return a list of 'TpuDevice'
    devices = jax.devices()
    print("JAX saw these devices:", devices)

    if any("tpu" in str(d).lower() for d in devices):
        print("✅ TPU is active and working!")
except Exception as e:
    print(f"❌ Hardware error: {e}")

JAX saw these devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)]
✅ TPU is active and working!


In [ ]:
import jax

# Check available devices
devices = jax.devices()
print(f"Connected to {len(devices)} TPU cores:")
for d in devices:
    print(f" - {d}")

Connected to 1 TPU cores:
 - TPU_0(process=0,(0,0,0,0))


In [ ]:
import jax.numpy as jnp
import jax

# Create large random matrices on the TPU
key = jax.random.PRNGKey(0)
x = jax.random.normal(key, (8192, 8192))
y = jax.random.normal(key, (8192, 8192))

# Time the multiplication
%timeit jnp.matmul(x, y).block_until_ready()

7.05 ms ± 28.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
import tensorflow as tf
import os

# 1. Initialize the TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

# 2. Run a calculation inside the 'strategy' scope
with strategy.scope():
    a = tf.random.normal([8192, 8192])
    b = tf.random.normal([8192, 8192])
    start = time.time()
    c = tf.matmul(a, b)
    end = time.time()
    print(f"TF TPU execution time: {end - start:.4f} seconds")